# An Implementation of Fast Agglomerative Hierarchical Clustering Algorithm Using Locality-Sensitive Hashing

## Walker Harrison and Lisa Lebovici (2018)

### Abstract

Agglomerative hierarchical clustering is a widely used clustering algorithm in which n data points are sequentially merged according to some distance metric from n clusters into a single cluster. While many different metrics exist to evaluate inter-cluster distance, single-linkage reigns among the most popular; according to this method, pairwise distances are computed between all points that do not share a cluster, and the clusters containing the two points with the shortest euclidean distance are joined in each iteration. However, single-linkage has some drawbacks, most noteworthy that it scales quadratically as data grows. As such, Koga, Ishibashi, and Watanbe propose using locality-sensitive hashing (hereafter LSH) as an approximation to the single-linkage method. Under LSH, points are hashed into buckets such that the distance from a given point p need only be computed to the subset of points with which it shares a bucket.

In this paper, we implement locality-sensitive hashing as a linkage method for agglomerative hierarchical clustering, optimize the code with an emphasis on reducing the time complexity of the creation of the hash tables, and apply the algorithm to a variety of synthetic and real-world data sets.

**Keywords**: agglomerative hierarchical clustering, locality-sensitive hashing, single-linkage, nearest neighbor search, dendrogram similarity, cophenetic correlation coefficient

**Github link**: https://github.com/lisalebovici/LSHLinkClustering

### 1 Background

With the growing ease in which data can be acquired in the modern age, techniques that can accurately represent this data are becoming increasingly important. In particular, unsupervised learning - that is, data which has no "ground truth" representation - finds itself at the center of many fields ranging from consumer marketing to computer vision to genetics. At its heart, the goal of unsupervised learning is pattern recognition. To this end, algorithms such as k-means and agglomerative hierarchical clustering have been sufficiently effective until now; but as the size of data continues to grow, scalability issues are becoming a bigger barrier to analysis and understanding.

Koga et al.'s *Fast Agglomerative Hierarchical Clustering Algorithm Using Locality-Sensitive Hashing* provides a faster approximation to single-linkage hierarchical agglomerative clustering for large data, which has a run time complexity of $O(n^2)$. The single-linkage method requires that the distances from every point to all other points not in its cluster are calculated on every iteration (of which there are $n-1$ iterations in a size $n$ data set), which becomes prohibitively expensive. In contrast, the primary advantage of LSH is that it reduces the number of distances that need to be computed as well as the the number of iterations that need to run, resulting in a linear run time complexity $O(nB)$ (where $B$ is the maximum number of points in a single hash table).

However, one consequence of the gain in run time is that LSH results in a coarser approximation of the data than single-linkage; while single-linkage necessarily creates clusters ranging from size $n$ to size $1$, LSH yields far fewer iterations of clusters. This granularity can be controlled by a parameter within the algorithm, but in general it is not expected to reproduce the single-linkage method exactly. Nonetheless, when granularity is not a primary concern, the improvements in terms of efficiency make it a worthwhile alternative.

We will proceed by describing the implementation of LSH for hierarchical clustering.

### 2 Description of Algorithm

Some important notation will first be defined:

- $n$: number of rows in data
- $d$: dimension of data
- $C$: least integer greater than the maximal coordinate value in the data
- $\ell$: number of hash functions
- $k$: number of sampled bits from a hashed value
- $r$: minimal distance between points required to merge clusters
- $A$: increase ratio of r on each iteration

LSH works in two primary phases: first, by creating a series of hash tables in which the data points are placed, and second, by computing the distances between a point and its "similar" points to determine which clusters should be merged.

**Phase 1: Generation of hash tables**. Suppose we have a d-dimensional point $x$. A unary function is applied to each coordinate value of $x$ such that $x$ 1s are followed by $C-x$ 0s. The sequence of 1s and 0s for all coordinate values of $x$ are then joined to form the hashed point of $x$. Some number $k$ bits are then sampled without replacement from the hashed point. For example, if $x$ is the point $(2, 1, 3)$ and $C = 4$, the hashed point would be $\underbrace{1100}_{2}\underbrace{1000}_{1}\underbrace{1110}_{3}$. If $k = 5$ and the indices $I = {5, 9, 2, 11, 8}$ are randomly sampled, then our resulting value would be $11110$. $x$ would thus be placed into the hash table for $11110$.

This hash function is applied to all points, and a point is added to the corresponding hash table if no other point in its cluster is already present (that is to say, hash tables, are uniqued by cluster). Another set of $k$ indices $I$ is then randomly sampled and the resulting hash function is applied to all of the points. This procedure is repeated $\ell$ times.

The intuition behind this step is as follows: a point $s$ that is very similar to $x$ is likely to have a similar hash sequence to $x$ and therefore appear in many of the same hash tables as $x$. However, for any given hash, there is no guarantee; for example, for $s = (1, 1, 3)$, the hashed point would be $\underbrace{1000}_{1}\underbrace{1000}_{1}\underbrace{1110}_{3}$ and the sampled value would be $11010$. In this cae, $s$ would not be in the same value as $x$ above. However, by applying $\ell$ hash functions to the data, we have some certainty that if $s$ is really close to $x$, it will share at least one hash table.

**Phase 2: Nearest neighbor search**. For each point $x$, we find all of the points that share at least one hash table with $x$ and are not currently in the same cluster as $x$. These are the similar points which are candidates to have their clusters merged with $x$'s cluster. The distances between $x$ and the similar points are computed, and for all points $p$ for which the euclidean distance between $x$ and $p$ is less than $r$, $p$'s clusters are merged with $x$'s cluster.

If there is more than one cluster remaining after the merges, the values for $r$ and $k$ are updated and then phases 1 and 2 are repeated. $r$ is increased (we now consider points that are a slightly further distance away than previously to merge further-away clusters) and $k$ is decreased (so that the hashed values become shorter and therefore more common). This continues until all points are in the same cluster, at which point the algorithm terminates.

#### LSH-Link Algorithm

> **Input:** Starting values for $\ell$, $k$, and $A$

> **Initialize**:

> > **if** $n < 500$

> > > sample $M = \{\sqrt{n}$ points from data}

> > > $r = \text{min dist}(p, q)$, where $p, q \in M$

> > **else**

> > > $r = \frac{d * C}{2 * (k + d)}\sqrt{d}$

> **while** num_clusters > 1:

> > **for** $i = 1,..,\ell$:

> > > $unary_C(x) = \underbrace{11...11}_{x}\underbrace{00...00}_{C-x}$

> > > sample $k$ bits from $unary_C(x)$

> > > **if** $x$'s cluster is not in hash table:

> > > > add $x$ to hash table

> > > repeat for $n$ points

> > **for** $p = 1,..,n$:

> > > S = {set of points that share at least one hash table with $p$}

> > > Q = {$q \in S$ s.t. $\text{dist}(p, q) < r$}

> > > merge $Q$'s clusters with $p$'s cluster

> > **if** num_clusters > 1:

> > > $r = A*r$

> > > $k = \frac{d * C}{2 * r}\sqrt{d}$